# droitGPT

# LLM: Qwen-1_8B-Chat-Int4

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MAX_NEW_TOKENS = 8192
"""
Source: 
- context length: https://huggingface.co/Qwen/Qwen-1_8B-Chat-Int4
"""
MODEL_ID = "Qwen/Qwen-1_8B-Chat-Int4"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,device_map="auto",trust_remote_code=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=MAX_NEW_TOKENS)
hf = HuggingFacePipeline(pipeline=pipe)

# Model performance - English
Answer is good enough

In [ ]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

# Model performance - French
Answer is good enough

In [ ]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Réponse: Réfléchissons étape par étape."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "Qu’est-ce que l’électroencéphalographie ?"

print(chain.invoke({"question": question}))

# Test LLM with specific context
Good performance so far

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """
Vous êtes un assistant spécialisé en codes juridiques français.

Utilisez les informations suivantes pour répondre à la question:
<context>
{context}
<context>

Question: {input}

Réponse: En utilisant le contexte, répondre à la question.
"""

prompt = PromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(hf, prompt)

In [ ]:
from langchain_core.documents import Document

specific_context = """Le Peuple français proclame solennellement son attachement aux Droits de l'Homme et aux principes de la souveraineté nationale tels qu'ils ont été définis par la Déclaration de 1789, confirmée et complétée par le préambule de la Constitution de 1946, ainsi qu'aux droits et devoirs définis dans la Charte de l'environnement de 2004.""" 

response = document_chain.invoke({
    "input": "Quels sont les éléments auxquels le Peuple français proclame solennellement son attachement dans ce texte ?",
    "context": [Document(page_content=specific_context)]
})

In [ ]:
print(response)

# Sentence Transformers
- Il faut trouver un modèle Sentence Transformers gratuit (donc pas de OpenAI Embeddings ni Ollama Embeddings)
- Ce modèle doit être pre-entrainé pour la comparaison semantique
- Ce modèle doit comprendre le français

Source https://www.sbert.net/docs/pretrained_models.html

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_id)

# Corpus
On va se limiter a 3 codes juridiques

## Clean Corpus

In [ ]:
import os
import re

data_folder = "data"
relevant_files = ["travail.md", "education.md", "electoral.md"]

clean_data_folder_name = "clean_data"

if not os.path.exists(clean_data_folder_name):
        os.makedirs(clean_data_folder_name)

for file in os.listdir(data_folder):
    if file in relevant_files:
        file_path = data_folder + "/" + file 
        clean_file_path = clean_data_folder_name + "/clean_" + file
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

            # Transform textdata with re here
            pattern1 = re.compile(r'---.*?---', re.DOTALL) # remove title and date 
            text = re.sub(pattern1, '', text)
            
            pattern2 = re.compile(r'#+.*\n') # remove Markdown titles
            text = re.sub(pattern2, '', text)

            pattern2 = re.compile(r'\*\*.*\*\*') # remove Article titles
            text = re.sub(pattern2, '', text)

            pattern3 = re.compile(r'<div.*</div>') # remove html content (simple approach, usually tables)
            text = re.sub(pattern3, '', text)

            pattern4 = re.compile(r'([:;])\n+') # form paragraphs
            text = re.sub(pattern4,lambda x: x.group(1) + ' ', text)

            with open(clean_file_path, "w", encoding="utf-8") as f:
                f.write(text)

In [ ]:
import os
import re

from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document


CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0

data_folder = "clean_data"
documents = []
for file in os.listdir(data_folder):
    filename = os.fsdecode(data_folder + "/" + file)
    loader = TextLoader(filename)
    documents += loader.load()

docs = []
for doc in documents:
    texts = re.split('\n+', doc.page_content)
    texts = [text for text in texts if text]
    for text in texts:
        docs.append(Document(page_content = text, metadata=doc.metadata))

In [ ]:
print(len(docs))
print()
print(docs[0])
print()
print(docs[100])

# Vector Database

In [ ]:
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(docs, embeddings)

# Querying the Database
On voit que les docs récuperés de la base de données sont un peu similaires au requête (modèle Embeddings n'est pas très grand)

In [ ]:
def print_context(docs_and_scores):
    for i, (doc, score) in enumerate(docs_and_scores):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [ ]:
query = "L'éducation des jeunes sourds"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

In [ ]:
query = "Contrat de travail à durée déterminée"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

In [ ]:
query = "Le mandat de conseiller départemental est incompatible"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

# Use Vector Database as context depending on the input/question

In [ ]:
def print_response(response):
    print("Answer:")
    print(response["answer"])
    print()
    print("Context:")
    contexts = response["context"]
    for i, doc in enumerate(contexts):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Test - Code Education

In [ ]:
question = "Est-ce que l'education est la première priorité nationale en France?"
response = retrieval_chain.invoke({"input": question})
print_response(response)

# Test - Code Electoral

In [ ]:
question = "Qu'est ce que tu connais sur le code electoral français?"
response = retrieval_chain.invoke({"input": question})
print_response(response)

# Test - Code Travail

In [ ]:
question = "Est-ce qu'une organisation syndicale de salariés peut agir devant une juridiction civile?"
response = retrieval_chain.invoke({"input": question})
print_response(response)